#### LeetCode - SQL - #1148

Refer [1148. Article Views I](https://leetcode.com/problems/article-views-i/description/)

Write a solution to find all the authors that viewed at least one of their own articles.

Return the result table sorted by id in ascending order.

In [29]:
data_articles = [[1, 3, 5, '2019-08-01'], [1, 3, 6, '2019-08-02'], [2, 7, 7, '2019-08-01'], [2, 7, 6, '2019-08-02'], [4, 7, 1, '2019-07-22'], [3, 4, 4, '2019-07-21'], [3, 4, 4, '2019-07-21']]
columns_articles = ['article_id', 'author_id', 'viewer_id', 'view_date']
schema_articles = {'article_id':'Int64', 'author_id':'Int64', 'viewer_id':'Int64', 'view_date':'datetime64[ns]'}

In [ ]:
from pyspark.sql import SparkSession

spark_context = SparkSession.builder.appName('LeetCode SQL').getOrCreate()

In [ ]:
# Create Spark Dataframe using Pandas
import pandas

pandas_df_articles = pandas.DataFrame(data=data_articles, columns=columns_articles).astype(schema_articles)

df_articles = spark_context.createDataFrame(pandas_df_articles)

df_articles.printSchema()
df_articles.show(5)

In [ ]:
# Create Spark Dataframe using PySpark functions - 1
from pyspark.sql import functions as F

schema_articles = 'article_id long, author_id long, viewer_id long, view_date_str string'

df_articles = spark_context.createDataFrame(data=data_articles, schema=schema_articles)
df_articles = df_articles.withColumn('view_date', F.to_date(df_articles.view_date_str))

df_articles.printSchema()
df_articles.show(5)

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema_articles = StructType([
    StructField('article_id', IntegerType(), False),
    StructField('author_id', IntegerType(), False),
    StructField('viewer_id', IntegerType(), False),
    StructField('view_date_str', StringType(), False),
])

df_articles = spark_context.createDataFrame(data=data_articles, schema=schema_articles)

df_articles = df_articles.select(
    df_articles.article_id,
    df_articles.viewer_id,
    df_articles.author_id,
    F.to_timestamp(df_articles.view_date_str).alias('view_date')
)

df_articles.printSchema()
df_articles.show(5)

In [ ]:
df_articles \
    .filter(df_articles.author_id == df_articles.viewer_id) \
    .select(df_articles.author_id) \
    .show()